In [2]:
from datasets import load_dataset
import os
os.environ['https_proxy'] = 'http://10.14.30.39:7890'
ds = load_dataset("fancyzhx/ag_news")
ds

Using the latest cached version of the dataset since fancyzhx/ag_news couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/fancyzhx___ag_news/default/0.0.0/eb185aade064a813bc0b7f42de02595523103ca4 (last modified on Thu Nov 21 01:50:51 2024).


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [3]:
train_ds = ds['train']
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [4]:
def map_len_summary(example):
    example['len_sum']=len(example['text'])
    return example

train_ds = train_ds.map(map_len_summary,num_proc=48)


len_mate_data = train_ds['len_sum']


import pandas as pd

def calculate_quartiles_with_pandas(data):
    """
    使用 Pandas 计算数据的四分位数
    :param data: 数据列表或数组
    :return: 第 1 四分位数、第 2 四分位数（中位数）、第 3 四分位数
    """
    series = pd.Series(data)
    q1 = series.quantile(0.25)  # 第 1 四分位数
    q2 = series.quantile(0.50)  # 中位数
    q3 = series.quantile(0.75)  # 第 3 四分位数
    return q1, q2, q3

# 示例用法
data = len_mate_data
q1, q2, q3 = calculate_quartiles_with_pandas(data)
print(f"第 1 四分位数: {q1}, 中位数: {q2}, 第 3 四分位数: {q3}")
print('mean:',sum(len_mate_data)/len(len_mate_data))

第 1 四分位数: 196.0, 中位数: 232.0, 第 3 四分位数: 266.0
mean: 236.477525


In [5]:
train_ds = train_ds.filter(lambda x: x['len_sum']>150, num_proc=48)
train_ds = train_ds.filter(lambda x: x['len_sum']<250, num_proc=48)
train_ds

Dataset({
    features: ['text', 'label', 'len_sum'],
    num_rows: 68776
})

In [6]:
from datasets import Dataset

train_ds_shu = train_ds.shuffle(seed=42)
dataset_used_A_train = train_ds_shu.select(range(0,20000))['text']
dataset_used_A_test = train_ds_shu.select(range(20000,22000))['text']
dataset_used_B_train_mem = train_ds_shu.select(range(22000,32000))['text']
dataset_used_B_train_non = train_ds_shu.select(range(32000,42000))['text']
dataset_used_B_test = train_ds_shu.select(range(42000,44000))['text']


import sys
sys.path.append('..')
from Data.utils import save_json,load_json

path = '../Data/raw/phishing/agnews.json'

save_json({
    'A':{
        'train':dataset_used_A_train,
        'test':dataset_used_A_test
    },
    'B':{
        'train':dataset_used_B_train_mem,
        'test':dataset_used_B_test,
        'train_non':dataset_used_B_train_non
    }
},path)